In [5]:
import PyPDF2
import json

def main(file_path, output_path):
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        num_pages = len(reader.pages)
        all_text = ''
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text()

    # Process `all_text` to generate your JSON data
    # For example, convert it to a dictionary or list
    
    # Example: Converting to a dictionary
    json_data = {
        "text": all_text,
        "pages": num_pages
    }

    # Writing JSON data to file
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

    print(f"Data successfully written to {output_path}")

if __name__ == "__main__":
    input_pdf = "Biology2e-WEB_Excerpt.pdf"  # Replace with your PDF file path
    output_json = "output1.json"  # Replace with your desired output file path
    main(input_pdf, output_json)


Data successfully written to output1.json


In [1]:
pip install Pillow


Note: you may need to restart the kernel to use updated packages.


In [5]:
import fitz  # PyMuPDF
import json
from pydantic import BaseModel
from typing import List, Optional
from PIL import Image
from io import BytesIO
import base64

class ImageModel(BaseModel):
    page_number: int
    image_index: int
    image_base64: str

class PageModel(BaseModel):
    page_number: int
    text: str
    images: Optional[List[ImageModel]]

class PDFModel(BaseModel):
    file_name: str
    pages: List[PageModel]

def extract_text_and_images(pdf_path: str) -> PDFModel:
    doc = fitz.open(pdf_path)
    pdf_data = PDFModel(file_name=pdf_path, pages=[])

    for page_number in range(len(doc)):
        page = doc[page_number]
        text = page.get_text("text")
        images = []

        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]

            # Convert image to base64
            img_base64 = base64.b64encode(image_bytes).decode('utf-8')
            images.append(ImageModel(page_number=page_number, image_index=img_index, image_base64=img_base64))

        page_data = PageModel(page_number=page_number, text=text, images=images)
        pdf_data.pages.append(page_data)

    return pdf_data

# Specify the PDF file path
pdf_file_path = 'Biology2e-WEB_Excerpt.pdf'

# Extract data and convert to JSON
pdf_data = extract_text_and_images(pdf_file_path)

# Convert to JSON using the json module
pdf_dict = pdf_data.dict()
pdf_json = json.dumps(pdf_dict, indent=4)

# Adding line spaces between different sections in the JSON string
pdf_json_with_spaces = pdf_json.replace('},', '},\n')

# Save JSON to a file
with open('output.json', 'w') as f:
    f.write(pdf_json_with_spaces)

print("PDF data has been successfully converted to JSON with line spaces and saved as 'output.json'.")

PDF data has been successfully converted to JSON with line spaces and saved as 'output.json'.


c:\users\dimple\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:53: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/


In [8]:
import fitz  # PyMuPDF
import json
from pydantic import BaseModel
from typing import List, Optional

class ImageModel(BaseModel):
    page_number: int
    image_index: int
    location: str  # Assuming location is a string representing the file path or URL

class PageModel(BaseModel):
    page_number: int
    text: str
    images: Optional[List[ImageModel]]

class PDFModel(BaseModel):
    file_name: str
    pages: List[PageModel]

def extract_text_and_images(pdf_path: str) -> PDFModel:
    doc = fitz.open(pdf_path)
    pdf_data = PDFModel(file_name=pdf_path, pages=[])

    for page_number in range(len(doc)):
        page = doc[page_number]
        text = page.get_text("text")
        images = []

        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_location = f'image_{page_number}_{img_index}.png'  # Example: Generating a location string

            # Save the image to a file (optional)
            with open(image_location, 'wb') as f:
                f.write(base_image['image'])

            images.append(ImageModel(page_number=page_number, image_index=img_index, location=image_location))

        page_data = PageModel(page_number=page_number, text=text, images=images)
        pdf_data.pages.append(page_data)

    return pdf_data

# Specify the PDF file path
pdf_file_path = 'Biology2e-WEB_Excerpt.pdf'

# Extract data
pdf_data = extract_text_and_images(pdf_file_path)

# Convert to JSON using the json module
pdf_dict = pdf_data.dict()
pdf_json = json.dumps(pdf_dict, indent=4)

# Adding line spaces between different sections in the JSON string
pdf_json_with_spaces = pdf_json.replace('},', '},\n')

# Save JSON to a file
with open('output.json', 'w') as f:
    f.write(pdf_json_with_spaces)

print("PDF data has been successfully converted to JSON with line spaces and saved as 'output.json'.")


PDF data has been successfully converted to JSON with line spaces and saved as 'output.json'.


c:\users\dimple\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:52: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
